### Entrenamiento del modelo de YOLOv8 con el dataset de imagenes etiquetadas

#### Instalación de la libreria ultralutics proveedora de YOLO


In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.3/896.3 kB 47.8 MB/s eta 0:00:00


#### Importacion de otras librerias necesarias para el proceso

In [2]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.6
    Uninstalling tqdm-4.66.6:
      Successfully uninstalled tqdm-4.66.6


#### Conexión con la unidad de drive donde estan las imagenes

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Definición de las rutas donde se van a alojar las imagenes dentro del entorno del colab para el entrenamiento

In [4]:
train_path_img = "./yolo_data/images/train"
train_path_label = "./yolo_data/labels/train"
val_path_img = "./yolo_data/images/val"
val_path_label = "./yolo_data/labels/val"
test_path = "./yolo_data/test"

#### División del set de datos en entrenamiento y prueba

In [5]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.png',f"{train_path_img}/" + filex + '.png' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".png", f"{train_path_img}/" + filex + '.png')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.png', f"{val_path_img}/" + filex + '.png' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
# train_test_split('/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/data/') ### without negative images
#train_test_split('/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/data/','/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/negativas/') ### if you want to feed negative images

train_test_split('/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/data/','/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/negativas/')


------ PROCESS STARTED -------
--- This folder has a total number of 123 images---


  0%|          | 0/99 [00:00<?, ?it/s]

------ Training data created with 80% split 99 images -------


  0%|          | 0/9 [00:00<?, ?it/s]

------ Total  9 negative images added to the training data -------
------ TOTAL Training data created with 108 images -------


  0%|          | 0/24 [00:00<?, ?it/s]

------ Testing data created with a total of 24 images ----------
------ TASK COMPLETED -------


Importación de la libreria ultralytics y verificación de su funcionamiento y los recursos que utilizará

In [6]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.0/112.6 GB disk)


#### Entrenamiento del modelo en modo deteccion con el modelo preentrenado yolov8n.pt 300 épocas y tamaño de lote 1

In [7]:
#entreno con 50 epocas
!yolo task=detect mode=train model=yolov8n.pt data=/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/dataset.yaml epochs=100 imgsz=640 batch=6 project=/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/training_results name=controlPastaTermica

#Mejora para el entrenar el modelo

#!yolo task=detect mode=train modelo=ylov8s.pt data=/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/dataset.yaml epochs=300 imgsz=640 batch=12 project=/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/training_results name=controlPastaTermica save_period=50

#model=yolov8s.pt  mejor modelomas robusto
#model=yolov8n.pt  modelo por defecto el mas pequeño



#!yolo task=detect mode=train model=yolov8n.pt data=/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/dataset.yaml epochs=300 imgsz=640 batch=6 project=/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/training_results name=controlPastaTermica


100% 6.25M/6.25M [00:00<00:00, 357MB/s]
Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/dataset.yaml, epochs=100, time=None, patience=100, batch=6, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/training_results, name=controlPastaTermica7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, a

#### Carga del modelo y exportacion en formato onnx (optimizado para uso en CPU)

In [8]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/training_results/controlPastaTermica/weights/best.pt")  # load a custom trained model

# Exportar el modelo
model.export(format="onnx")

Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/training_results/controlPastaTermica/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 303.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 188.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.5/291.5 MB 302.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 136.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 286.5 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 17.7s, installed 3 packages: ['onnx>=1.12

'/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/training_results/controlPastaTermica/weights/best.onnx'

#### Deteccion en los videos de prueba

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/training_results/controlPastaTermica8/weights/best.pt conf=0.77 source=/content/drive/MyDrive/PracticaProfesionalizanteII/YOLOv8/videos


Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 940, in entrypoint
    model = YOLO(model, task=task)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/models/yolo/model.py", line 23, in __init__
    super().__init__(model=model, task=task, verbose=verbose)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 145, in __init__
    self._load(model, task=task)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 285, in _load
    self.model, self.ckpt = attempt_load_one_weight(weights)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py", line 910, in attempt_load_one_weight
    ckpt, weight = torch_safe_load(weight)  # load ckpt
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py", line 837, in torch_safe_load
    ckpt = torch.load(fi

#### Copia de los videos con las predicciones realizadas desde el entorno de colab a una carpeta de Drive

In [ ]:
!cp -r /content/runs/detect/predict/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/output/

cp: missing destination file operand after '/content/runs/detect/predict/content/drive/MyDrive/Practicas_Profesionalizantes_II/DatosDeModelo/YOLOv8/output/'
Try 'cp --help' for more information.
